In [ ]:
#!pip install openmeteo-requests
#!pip install requests-cache retry-requests numpy pandas

In [1]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 47.4239,
	"longitude": 9.3748,
	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "rain_sum", "snowfall_sum"],
	"timezone": "Europe/Berlin"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
daily_rain_sum = daily.Variables(3).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(4).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s"),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["rain_sum"] = daily_rain_sum
daily_data["snowfall_sum"] = daily_snowfall_sum

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)

Coordinates 47.439998626708984°E 9.399999618530273°N
Elevation 679.0 m asl
Timezone b'Europe/Berlin' b'CET'
Timezone difference to GMT+0 3600 s
                 date  weather_code  temperature_2m_max  temperature_2m_min  \
0 2023-11-11 23:00:00          81.0              8.5695              1.1195   
1 2023-11-12 23:00:00          80.0             14.2695              7.8195   
2 2023-11-13 23:00:00          80.0             12.9695              9.3140   
3 2023-11-14 23:00:00          80.0              9.0640              4.9140   
4 2023-11-15 23:00:00          63.0              7.4640              3.7140   
5 2023-11-16 23:00:00          85.0              6.7140              3.0175   
6 2023-11-17 23:00:00          85.0              7.4175              2.9175   

    rain_sum  snowfall_sum  
0  13.900001          0.00  
1  17.300001          0.00  
2  28.400000          0.00  
3   0.800000          0.00  
4  14.100000          0.00  
5  15.500000          0.07  
6   1.000000        

In [2]:
daily_dataframe.to_csv("C:/Users/rita_/OneDrive/Dokumente/git/parcaster/data/scraping/weather/forecast_weather/forecast_weather.csv"
                       , sep=","
                       , index=False)